In [1]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
#%% load the data
data_dir = os.getcwd() + '\\Data\\'
image_dir = data_dir + 'images\\rev\\' 

# total count of samples
sample_count = 1584

# load images
images = []
for image_label in range(sample_count):
    image = mpimg.imread(image_dir + str(image_label + 1) + '.jpg')
    images.append(image)
    
images = np.array(images)

In [3]:
# load attributes 
train_data = pd.read_csv(data_dir + 'train.csv')
train_ids = train_data.ix[:, 'id']
train_labels = train_data.ix[:, 'species']
train_margins = train_data.iloc[:, 1:66]
train_shapes = train_data.iloc[:, 66: 130]
train_textures = train_data.iloc[:, 130: 194]

In [4]:
TOTAL_COUNT = len(train_data) # training image count in input csv file
IMAGE_WIDTH  = images[0].shape[0] # input image width
IMAGE_HEIGHT = images[0].shape[1] # input image height
OUTPUT_TYPE_COUNT = len(list(set(train_labels)))

In [5]:
train_images = np.reshape(images[train_ids - 1], (TOTAL_COUNT, IMAGE_WIDTH, IMAGE_HEIGHT, -1))

train_label_ids = []
label_ids = np.array(list(set(train_labels)))
for i in range(len(train_labels)):
    train_label_ids.append([np.where(label_ids == train_labels[i])[0][0]])
    
train_label_ids = np.array(train_label_ids)
train_label_bits = np.zeros(shape=(TOTAL_COUNT,OUTPUT_TYPE_COUNT))

for i in range(OUTPUT_TYPE_COUNT):
    train_label_bits[i][train_label_ids[i]] = 1

In [6]:
import tensorflow as tf
import math

In [7]:
# parameters
BATCH_SIZE = 100
ITERATION_COUNT = 1000
ACCURACY_CHECK = 20

In [8]:
# the batch size equals to BATCH_SIZE it will
# starts from 0 to until the last index of
# batch equals to TOTAL_COUNT
BATCH_COUNTER = 0

def next_batch(all_images, all_labels):
    
    # accessing BATCH_COUNTER inside function
    global BATCH_COUNTER
    
    # check if exceeds to the last index then reset batch_counter
    if BATCH_COUNTER == len(all_labels) / BATCH_SIZE:
        BATCH_COUNTER = 0

    # setting first and last index of data
    index_from = BATCH_COUNTER * BATCH_SIZE
    index_to   = (BATCH_COUNTER + 1) * BATCH_SIZE
    
    # loading a batch of training images and labels
    image_batch = all_images[index_from:index_to]
    label_batch = all_labels[index_from:index_to]
    
    # incrementing batch_counter for the next iteration
    BATCH_COUNTER += 1 
    
    return image_batch, label_batch;

In [ ]:
def train_model(images, labels):
    
    image_width  = images[0].shape[0]
    image_height = images[0].shape[1]
    
    C0 = 1   # input channel count
    C1 = 4   # convolutional network channel 1 count 
    C2 = 8   # convolutional network channel 2 count
    C3 = 12  # convolutional network channel 3 count
    C4 = 200 # fulley connected layer size
    C5 = OUTPUT_TYPE_COUNT  # output count 
    
    # weights
    W1 = tf.Variable(tf.truncated_normal([5, 5, C0, C1], stddev = 0.1)) 
    W2 = tf.Variable(tf.truncated_normal([5, 5, C1, C2], stddev = 0.1))
    W3 = tf.Variable(tf.truncated_normal([4, 4, C2, C3], stddev = 0.1))
    W4 = tf.Variable(tf.truncated_normal([7 * 7 * C3, C4], stddev = 0.1))
    W5 = tf.Variable(tf.truncated_normal([C4, C5], stddev = 0.1))
    
    # biases
    B1 = tf.Variable(tf.ones([C1]) / 10)
    B2 = tf.Variable(tf.ones([C2]) / 10)
    B3 = tf.Variable(tf.ones([C3]) / 10)
    B4 = tf.Variable(tf.ones([C4]) / 10)
    B5 = tf.Variable(tf.ones([C5]) / 10)

    # model
    stride = 1  # output is 28x28
    X   = tf.placeholder(tf.float32, shape=[None, image_width, image_height, C0])
    Y1  = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    stride = 2  # output is 14x14
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    stride = 2  # output is 7x7
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)
    
    # reshape the output from the third convolution for the fully connected layer
    YY = tf.reshape(Y3, shape=[-1, 7 * 7 * C3])
    Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
    Ylogits = tf.matmul(Y4, W5) + B5
    Y = tf.nn.softmax(Ylogits)
                      
    # paceholder for learing_rate variable
    lr = tf.placeholder(tf.float32)
    
    # placeholder for correct answers
    Y_ = tf.placeholder(tf.float32, [None, C5])
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels = Y_)
    cross_entropy = tf.reduce_mean(cross_entropy) * 100
    
    # accuracy of the trained model, between 0 (worst) and 1 (best)
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # training step, the learning rate is a placeholder
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

    # initialisation
    tf.set_random_seed(0)
    init = tf.global_variables_initializer()

    # start computation
    sess = tf.Session()
    sess.run(init)
    
    # preparing return values
    counter = 0 
    upper_bound = int(ITERATION_COUNT / ACCURACY_CHECK)
    train_accuracy = np.zeros(upper_bound)
    train_loss = np.zeros(upper_bound)
    test_accuracy = np.zeros(upper_bound)
    test_loss = np.zeros(upper_bound)
    learning_rates = np.zeros(1)
    
    for i in range(ITERATION_COUNT):
        print(i)
        # learning rate decay
        max_learning_rate = 0.003
        min_learning_rate = 0.0001
        decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
        
        # load batch of images and correct answers
        batch_X , batch_Y = next_batch(images,labels)
        
        # train
        sess.run(train_step, feed_dict = {X: batch_X, Y_: batch_Y, lr: learning_rate})
        
        if i % ACCURACY_CHECK == 0:

            a , c = sess.run([accuracy, cross_entropy], feed_dict = {X: batch_X, Y_: batch_Y})
            train_accuracy[counter] = a
            train_loss[counter] = c
            
            #test_data = {X: mnist_data.test.images, Y_: mnist_data.test.labels}
            #pred, a , c = sess.run([Y, accuracy, cross_entropy], feed_dict = test_data)
            #predictions = tf.argmax(pred,1).eval(session=sess)
            #print('{}- Accuracy: {}, Learning Rate: {}'.format(i, a, learning_rate))
            #test_accuracy[counter] = a
            #test_loss[counter] = c
            
            counter += 1
    
    #learning_rates[0] = learning_rate
    #acc = accuracy.eval(session = sess, feed_dict = {X: mnist_data.test.images, Y_: mnist_data.test.labels})
    #print('Learning Rate: {}, Accuracy: {}'.format(learning_rate, acc))
    sess.close()
    
    return train_accuracy, train_loss, test_accuracy, test_loss, predictions, learning_rates;

In [ ]:
train_accuracy = []
train_loss = []
learning_rates = []

train_images1 = np.reshape(train_images, (990, IMAGE_WIDTH, IMAGE_HEIGHT, -1))

%time lr, ta, tl = train_model(train_images1, train_label_bits)

acc_check = np.arange(0, len(lr))
learning_rates.append(lr)
train_accuracy.append(ta)
train_loss.append(tl)

0
